In [23]:
import yaml
import pandas as pd
from pathlib import Path
from config import client
import requests
from IPython.display import display, Markdown

from config import column_names

In [26]:
# load markdown from github repository https://github.com/moritzvitt/moritzProjekt/blob/markdown/system_japanese.md
class BaseAnkiDeckGenerator:
    def __init__(self, df: pd.DataFrame):
        self.ai_input_df = self.df[['short_phrase', 'short_translation', 'word']]


    # @log_io

def load_markdown():
    url1 = "https://raw.githubusercontent.com/moritzvitt/moritzProjekt/markdown/system_japanese.md"
    url2 = "https://raw.githubusercontent.com/moritzvitt/moritzProjekt/markdown/add_furigana.md"
    
    response1 = requests.get(url1)
    markdown1 = response1.text

    response2 = requests.get(url2)
    markdown2 = response2.text

    merged_markdown = markdown1 + "\n" + markdown2
    return merged_markdown

# load csv from test_dataframes 

# load dataframe from csv
df = pd.read_csv('../test_dataframes/japanese_items/short_items.csv', delimiter='\t', encoding='utf-8')
# print(df.head())
# print(df.shape)
# # load column_names from config.py
# df.columns = column_names

# dfa = df[[
#         "Word", 
#         "Context",
#         "Context machine translation",
#         "Context human translation",
#         ]]

# # select only the first 10 rows
# dfa = dfa.head(10)
# # dfa to csv
# dfa.to_csv('../test_dataframes/japanese_items/short_items.csv', sep='\t', encoding='utf-8', index=False)
    

# run 
def run():
    markdown = load_markdown()
    display(Markdown(markdown))
    # also safe the markdown to a new file
    with open("moin.md", "w") as file:
        file.write(markdown + df.to_markdown())


if __name__ == "__main__":
    run()

# IDENTITY and PURPOSE

You are a professional language teacher, dedicated to helping a student learn a language. Your task is to provide concise, relevant information for sentence-word Anki flashcards, ensuring the student can effectively study vocabulary. 

# TOOlS

You rely mainly on your capability as LLM to predict the next string of characters. You don't need to analyse the table or anything. 

# INPUT:

You will work with a table of sentence-word pairs from the Google extension 'LanguageReactor', containing the following columns:

- 'Word'
- 'Context'
- 'Context machine translation'
- ('Context human translation')

The 'Context' column contains a sentence in the target language, that the student wants to learn. The 'Word' column contains one word that appears in the 'Context' sentence.

# Steps

1. ### Clean the data and check for parsing errors

   Have a look at the table I provided you with. Don't use code for that, just rely on your prediction of characters as LLM.

   - Remove furigana in brackets and correct weird formatting from 'Word' and 'Context'. However, pay attention that 'Word' always appears in 'Context'.
   - check each row to ensure the 'Context' sentence is correctly parsed. The 'Word' should include the entire vocabulary word, not just a fragment. Pay particular attention to languages like Japanese, where parsers may miss the whole verb or expression. Also, check the 'Context machine translation' to see if the 'Word' makes sense in its 'Context'. If there is a parsing error and 'Word' is incomplete, adjust 'Word' to match the vocabulary in 'Context'. Ensure 'Word' is formatted exactly as it appears in 'Context' (including capitalization, grammar, punctuation, and spelling errors if present).

2. ### Generate flashcard information

   To assist the student, generate the following information for each row:

   1. Two or more synonyms for 'Word' based on its 'Context'.
   2. Two or more translations for 'Word' based on its 'Context'.
   3. A simple 'Example sentence' using 'Word'.
   4. The translation of the 'Example sentence'.
   5. A brief explanation of 'Word' in its 'Context'.
   6. A short explanation of the grammar.

   When generating this information, stick to the following principles:

   1. Minimum Information Principle: Formulate the material in the simplest possible way without losing essential information. Simplicity should not mean skipping difficult parts. That means you can safely leave out conjunctions like 'or', 'and' and you don't need to say: 現実 means 'reality' or 'actuality'. Instead just say: 現実: reality, actuality.
   2. Optimize Wording: Ensure the wording is precise and efficient to trigger the correct response quickly. This will reduce errors, increase specificity, reduce response time, and enhance concentration.

3. ### FURIGANA for EVERY JAPANESE word! THIS IS EXTREMELY IMPORTANT!

   Add furigana in square brackets behind EACH kanji word and add a space before each kanji word The space before each kanji word is EXTREMELY important! Double check, – no – triple check that. Furigana should be added to all the columns containing Japanese, also to those containg a mix of Japanese and English.

   - 私[わたし]は 大学生[だいがくせい]です。
   - Attention: this would be wrong, as '事', '時間', '代' and '守' lack a blank space before. ハク 龍[りゅう] あなたのした事[こと]は もうとがめません その代[か]わり その 子[こ]を しっかり守[まも]るんだよ さあ 坊[ぼう]やたち お帰[かえ]りの時間[じかん]だよ.
   - Same thing here, spaces missing before '代わり' and '行って': 私[わたし]の代[か]わりに行[い]ってください。

4. ### Output

   - Output the generated information as a Markdown table, including the column names as headers.
   - Do not include warnings or notes in the output—only the requested sections.
   - Do not include additional information like 'here is the markdown table' or anything else. The only thing I want to have is the markdown table.


# EXAMPLES

This is how the information you generate should look like:

| Word       | Context                                                                                                                                                                 | Machine Translation                                                                                            | Synonyms                       | Translations        | Example sentence                             | Example sentence translation (German) | Explanation                                        |           Grammar explanation           | Additional Notes for chatGPT                                                                      |
| ------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------- | -------------------------------- | --------------------- | ---------------------------------------------- | --------------------------------------- | ---------------------------------------------------- | :---------------------------------------: | --------------------------------------------------------------------------------------------------- |
| 子[こ]     | "ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり 守[まも]るんだよ さあ 坊[ぼう]やたち お 帰[かえ]りの 時間[じかん]だよ" | "I won't do what you've done. Instead, we're going to protect that child. Come on, boy, it's time to go home." | 子ども[こども]、幼児[ようじ]   | child, kid          | その 子[こ]はかわいいです。                  | Das Kind ist süß.                   | child                                              |  子[こ] means "child." Used as a noun.  | child is a simple word, don't make it complicated. Just give me the translation as 'Explanation'. |
| 代[か]わり | "ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり守るんだよ さあ 坊[ぼう]やたち お 帰[かえ]りの 時間[じかん]だよ"        | "I won't do what you've done. Instead, we're going to protect that child. Come on, boy, it's time to go home." | 代理[だいり]、替[か]わり       | instead, substitute | 彼[かれ]の 代[か]わりに 行[い]ってください。 | Gehen Sie bitte an seiner Stelle.     | instead, 'instead of doing ...'                    | 代わり means "instead." Used as a noun. |                                                                                                   |
| 事[こと]   | "おばあちゃん ハク生きてた ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり 守[まも]るんだよ"                            | "Grandma, Haku was alive. I won't do what you've done. Instead, we're going to protect that child."            | 物事[ものごと]、事柄[ことがら] | thing, matter       | その 事[こと]は 難[むずか]しいです。         | Diese Sache ist schwierig.            | action, deed, 'your deeds'                         |   事 means "thing" or "matter." Noun.   | '事' is a simple word, don't make it complicated. Just give me the translation as 'Explanation'.  |
| 生[い]き   | "グッドタイミングね おばあちゃん ハク生きてた ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません"                                                                 | "Good timing. Grandma, Haku was alive. I won't do what you've done."                                           | 生[い]きる、存在[そんざい]する | alive, living       | 彼[かれ]はまだ 生[い]きています。            | Er lebt noch.                         | 生きる: to live, u-verb, 生きて(い)た = was living |      生きる means "to live." Verb.      | This is a verb                                                                                    |
| タイミング | "よかった グッドタイミングね おばあちゃん ハク生きてた"                                                                                                                 | "It was good Good timing. Grandma, Haku was alive."                                                            | 時期[じき]、機会[きかい]       | timing, opportunity | 今[いま]がいい タイミングです。              | Jetzt ist ein guter Zeitpunkt.        | 'Timing', English loanword                         |    タイミング means "timing." Noun.    | Just give me the translation for the katakana, as this is a woard every english speaker knows.    |

This is the table:

Word	Context	Context machine translation	Context human translation
程度	"ただし入れすぎると卵の味よりマヨネーズの味が勝ってしまうので
最小限つなぎ程度にして
最後に塩と胡椒で味を整えてください"	"But if you put too much in it, it tastes like mayonnaise.
We're going to make sure that we have the least amount of connections.
Finally, add some salt and pepper."	"But don't go overboard with it, or the mayonnaise will overpower the taste of eggs.
Keep the mayonnaise to a minimum,
and add salt and pepper to round out the flavors."
最小限	"ただし入れすぎると卵の味よりマヨネーズの味が勝ってしまうので
最小限つなぎ程度にして
最後に塩と胡椒で味を整えてください"	"But if you put too much in it, it tastes like mayonnaise.
We're going to make sure that we have the least amount of connections.
Finally, add some salt and pepper."	"But don't go overboard with it, or the mayonnaise will overpower the taste of eggs.
Keep the mayonnaise to a minimum,
and add salt and pepper to round out the flavors."
刻ん	"さすがにまだ卵サンドは食べられないみたいだけど
まずゆで卵を適度に刻んでください
マヨネーズって何でも合いますよね"	"I can't seem to eat an egg sandwich yet.
First of all, you need to mark the eggs in moderation.
I mean, mayonnaise is good for anything."	"But he hasn't had egg sandwiches since then.
First, dice the boiled eggs.
Mayonnaise goes well with everything."
適度	"さすがにまだ卵サンドは食べられないみたいだけど
まずゆで卵を適度に刻んでください
マヨネーズって何でも合いますよね"	"I can't seem to eat an egg sandwich yet.
First of all, you need to mark the eggs in moderation.
I mean, mayonnaise is good for anything."	"But he hasn't had egg sandwiches since then.
First, dice the boiled eggs.
Mayonnaise goes well with everything."
さすが	"その後も毎日中島くんは新聞を配っている
さすがにまだ卵サンドは食べられないみたいだけど
まずゆで卵を適度に刻んでください"	"Ever since then, I've been delivering newspapers every day.
I can't seem to eat an egg sandwich yet.
First of all, you need to mark the eggs in moderation."	"Nakajima still delivers the papers every day.
But he hasn't had egg sandwiches since then.
First, dice the boiled eggs."
現実	"君が吐いた息を吸って後悔となってる
現実は映画のようにはうまくいかない
その後も毎日中島くんは新聞を配っている"	"I'm breathing in your vomit, and now I regret it.
Reality doesn't work like the movies.
Ever since then, I've been delivering newspapers every day."	"MAINICHI SHINBUN
Life isn't like the movies. Sometimes you don't get a happy ending.
Nakajima still delivers the papers every day."
浮かぶ	"君が吐いた白い息が今ゆっくり風に乗って
空に浮かぶ雲の中に少しずつ消えてゆく
遠く高い空の中で手を伸ばす白い雲"	"The white breath you spewed is now moving slowly in the wind.
It disappears into the clouds.
White clouds reaching high into the sky."
吐い	"自分が本気で惚れた女 安く見るもんじゃないよ
君が吐いた白い息が今ゆっくり風に乗って
空に浮かぶ雲の中に少しずつ消えてゆく"	"I'm not looking for a woman that I really love.
The white breath you spewed is now moving slowly in the wind.
It disappears into the clouds."
本気	"誇りに思っていいんじゃないのかい
自分が本気で惚れた女 安く見るもんじゃないよ
君が吐いた白い息が今ゆっくり風に乗って"	"I don't think you should be proud.
I'm not looking for a woman that I really love.
The white breath you spewed is now moving slowly in the wind."	"You should be honored that she fell in love with you.
You loved her too, didn't you? Don't insult her by saying you didn't deserve her.
"
誇り	"そんな女に惚れられたってこと
誇りに思っていいんじゃないのかい
自分が本気で惚れた女 安く見るもんじゃないよ"	"I mean, he fell in love with her.
I don't think you should be proud.
I'm not looking for a woman that I really love."	"She didn't have to, but she did it anyway.
You should be honored that she fell in love with you.
You loved her too, didn't you? Don't insult her by saying you didn't deserve her."




# IDENTITY and PURPOSE

You function as a japanese parser. You're main purpose is to provide furigana for japanese texts, in square brackets [] behind each kanji word and a blank space before each kanji word. This is, because Anki, the flashcard software the student uses, only accepts this format.

# INPUT

- a table with information that the student needs to study vocabulary

# Steps

1. ### Add FURIGANA for EVERY JAPANESE word! THIS IS EXTREMELY IMPORTANT!

   Add furigana in square brackets behind EACH kanji word and add a space before each kanji word The space before each kanji word is EXTREMELY important! Double check, – no – triple check that. Furigana should be added to all the columns containing Japanese, also to those containg a mix of Japanese and English.


   - 私[わたし]は 大学生[だいがくせい]です。
   - Attention: this would be wrong, as '事', '時間', '代' and '守' lack a blank space before. ハク 龍[りゅう] あなたのした事[こと]は もうとがめません その代[か]わり その 子[こ]を しっかり守[まも]るんだよ さあ 坊[ぼう]やたち お帰[かえ]りの時間[じかん]だよ.
   - Same thing here, spaces missing before '代わり' and '行って': 私[わたし]の代[か]わりに行[い]ってください。
2. ### Output


   - Output the generated information as a Markdown table, including the column names as headers.
   - Do not include warnings or notes in the output—only the requested sections.
   - Do not include additional information like 'here is the markdown table' or anything else. The only thing I want to have is the markdown table.

# EXAMPLES

Here are some examples of how you should add the furigana!

| Word       | Context                                                                                                                                                                 |                                                                                                                | Synonyms                       | Example sentence                             | Explanation                                                |            Grammar explanation            |
| ---------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- | ------------------------------ | -------------------------------------------- | ---------------------------------------------------------- | :---------------------------------------: |
| 子[こ]     | "ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり 守[まも]るんだよ さあ 坊[ぼう]やたち お 帰[かえ]りの 時間[じかん]だよ" | "I won't do what you've done. Instead, we're going to protect that child. Come on, boy, it's time to go home." | 子ども[こども]、幼児[ようじ]   | その 子[こ]はかわいいです。                  | child                                                      |   子[こ] means "child." Used as a noun.   |
| 代[か]わり | "ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり守るんだよ さあ 坊[ぼう]やたち お 帰[かえ]りの 時間[じかん]だよ"        | "I won't do what you've done. Instead, we're going to protect that child. Come on, boy, it's time to go home." | 代理[だいり]、替[か]わり       | 彼[かれ]の 代[か]わりに 行[い]ってください。 | instead, 'instead of doing ...'                            |  代わり means "instead." Used as a noun.  |
| 事[こと]   | "おばあちゃん ハク生きてた ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません その 代[か]わり その 子[こ]を しっかり 守[まも]るんだよ"                            | "Grandma, Haku was alive. I won't do what you've done. Instead, we're going to protect that child."            | 物事[ものごと]、事柄[ことがら] | その 事[こと]は 難[むずか]しいです。         | action, deed, 'your deeds'                                 | 事[こと] means "thing" or "matter." Noun. |
| 生[い]き   | "グッドタイミングね おばあちゃん ハク生きてた ハク 龍[りゅう] あなたのした 事[こと]は もうとがめません"                                                                 | "Good timing. Grandma, Haku was alive. I won't do what you've done."                                           | 生[い]きる、存在[そんざい]する | 彼[かれ]はまだ 生[い]きています。            | 生[い]きる: to live, u-verb, 生[い]きて(い)た = was living |     生[い]きる means "to live." Verb.     |
| タイミング | "よかった グッドタイミングね おばあちゃん ハク生[い]きてた"                                                                                                             | "It was good Good timing. Grandma, Haku was alive."                                                            | 時期[じき]、機会[きかい]       | 今[いま]がいい タイミングです。              | 'Timing', English loanword                                 |     タイミング means "timing." Noun.     |


In [4]:
# load dataframe from csv
df = pd.read_csv('../test_dataframes/japanese_items/items.csv', delimiter='\t', encoding='utf-8')
print(df.head())
print(df.shape)
# load column_names from config.py
df.columns = column_names

dfa = df[[
        "Word", 
        "Context",
        "Context machine translation",
        "Context human translation",
        ]]

# select only the first 10 rows
dfa = dfa.head(10)
# dfa to csv
dfa.to_csv('../test_dataframes/japanese_items/short_items.csv', sep='\t', encoding='utf-8', index=False)

    WORD|胡椒|ja  Word        最後に塩と胡椒で味を整えてください  \
0   WORD|程度|ja  Word              最小限つなぎ程度にして   
1  WORD|最小限|ja  Word              最小限つなぎ程度にして   
2   WORD|刻む|ja  Word         まずゆで卵を適度に刻んでください   
3   WORD|適度|ja  Word         まずゆで卵を適度に刻んでください   
4  WORD|さすが|ja  Word  さすがにまだ卵サンドは食べられないみたいだけど   

                  Finally, add some salt and pepper.   胡椒 胡椒.1  Noun  \
0  We're going to make sure that we have the leas...   程度   程度  Noun   
1  We're going to make sure that we have the leas...  最小限  最小限  Noun   
2  First of all, you need to mark the eggs in mod...   刻ん   刻む  Verb   
3  First of all, you need to mark the eggs in mod...   適度   適度  Noun   
4           I can't seem to eat an egg sandwich yet.  さすが  さすが   Adv   

   Unnamed: 7                         pepper, black pepper  Netflix  ...  216  \
0         NaN                       degree, extent, amount  Netflix  ...  215   
1         NaN  minimum, minimum limit, minimum requirement  Netflix  ...  215   
2         NaN                

In [25]:
# Example of an OpenAI ChatCompletion request with stream=True and stream_options={"include_usage": True}

# a ChatCompletion request
response = client.chat.completions.create(
    model='gpt-4',
    messages=[
        {'role': 'user', 'content': "Can you give me a summary of the pros and cons of the OpenAI API?"},
    ],
    temperature=0,
    stream=True,
    # stream_options={"include_usage": True}, # retrieving token usage for stream response
)

for chunk in response:
    print(f"choices: {chunk.choices}\nusage: {chunk.usage}")
    # print(f"{chunk.content}")
    print("****************")

choices: [Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)]
usage: None


AttributeError: 'ChatCompletionChunk' object has no attribute 'content'

In [43]:
from openai import OpenAI
client = OpenAI()

stream = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': "Moritz is a great guy from Berlin"},
        # {'role': 'assistant', 'content': "I don't know that"},
        {'role': 'user', 'content': "who is Moritz?"}
    ],
    temperature=0,
    stream=True,
    stream_options={"include_usage": True}, # retrieving token usage for stream response
)

# print the content of the response

# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")

for chunk in stream:
    print(chunk)



ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', usage=None)
ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='Mor', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', usage=None)
ChatCompletionChunk(id='chatcmpl-9VIXMqyH2Vcb1bayBfZID39Crmnx7', choices=[Choice(delta=ChoiceDelta(content='itz', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717246452, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_319be4768e', 

In [23]:
from openai import OpenAI

client = OpenAI()

# List of messages to be sent to the chat completion endpoint
messages = [
    {"role": "user", "content": "Always answer briefly. My name is Moritz! What is your name?"},
    {"role": "user", "content": "Guess where I likely come from, based on my name"},
    {"role": "user", "content": "Please sum up this dialogue"},
]

# Create the chat completion stream
stream = client.chat.completions.create(
    model="gpt-4",  # Assuming you're using the latest GPT-4 model
    messages=messages,
    stream=True,
)

# Initialize an empty response container
full_response = ""

# Iterate over the stream and print each chunk
for chunk in stream:
    if 'choices' in chunk:
        delta = chunk['choices'][0]['delta']
        if 'content' in delta:
            full_response += delta['content']
            print(delta['content'], end="")

# Optionally, print the full response at the end
print("\n\nFull Response:\n", full_response)




Full Response:
 


In [15]:
from openai import OpenAI
import os
import nest_asyncio
import asyncio

client = OpenAI()
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize the OpenAI client with your API key


async def main():
    # Create a streaming compion
    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": "Always answer briefly. My name is Moritz! What is your name?"},
            {"role": "user", "content": "Guess where I likely come from, based on my name"},
            {"role": "user", "content": "Please sum up this dialogue"}
        ],
        stream=True
    )

    # Print the streaming response
    async for chunk in response:
        if 'choices' in chunk:
            choice = chunk['choices'][0]
            if 'delta' in choice and 'content' in choice['delta']:
                print(choice['delta']['content'], end="")

# Run the main function
asyncio.run(main())


TypeError: object Stream can't be used in 'await' expression

In [3]:
csv_file_path = Path("test_dataframes/italian_items/items_long.csv")

    # Load the csv file
df = pd.read_csv(csv_file_path, delimiter='\t')

with open('config/column_names.yaml', 'r') as file:
        data = yaml.safe_load(file)
        column_names = data['column_names']


df.columns = column_names

# select the columns word and long_sentence

df = df[['word', 'long_phrase']]
df.to_csv('test_dataframes/italian_items/items_long_changed.csv', sep='\t', index=False)



In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

# Composition

In [2]:
class Animal:
    def __init__(self):
        self.animal_type = "Generic Animal"

    def make_sound(self):
        print(f"{self.animal_type} sound")

class Dog:
    def __init__(self):
        self.animal_sound = "Woof!"
        self.animal = Animal()  # Create an Animal instance

    def make_sound(self):
        self.animal.make_sound()  # Call base class method directly
        print(self.animal_sound)

# Create objects and call methods
animal = Animal()
dog = Dog()

animal.make_sound()  # Output: Generic Animal sound
dog.make_sound()  # Output: Generic Animal sound
                #        Woof!


Generic Animal sound
Generic Animal sound
Woof!


In [1]:
class Animal:
    def __init__(self):
        self.animal_type = "Generic Animal"

    def make_sound(self):
        print(f"{self.animal_type} sound")

class Dog(Animal):
    def make_sound(self):
        self.animal_type = "Dog"  # Modify attribute before calling base class method
        super().make_sound()
        print("Woof!")

# Create objects and call methods
animal = Animal()
dog = Dog()

animal.make_sound()  # Output: Generic Animal sound
dog.make_sound()  # Output: Dog sound
                #        Woof!


Generic Animal sound
Dog sound
Woof!


In [7]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Markdown Table Example</title>
    <script src="https://cdn.jsdelivr.net/npm/marked@4.0.12/marked.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 40em;
        }
        table {
            width: 50%;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <div id="markdown-content"></div>

    <script>
        const markdownTable = `
            | Name   | Age | Occupation   |
            |--------|-----|--------------|
            | John   | 25  | Engineer     |
            | Alice  | 30  | Designer     |
            | Bob    | 22  | Developer    |
            `;

        document.getElementById('markdown-content').innerHTML = marked.parse(markdownTable);
    </script>
</body>
</html>


SyntaxError: invalid decimal literal (3130921398.py, line 18)